# Part 1: Single Layer Neural Network

In [12]:
%matplotlib inline

import numpy as np
import time

from keras import optimizers
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K

np.random.seed(0)

In [13]:
num_examples = 2000
num_classes = 10
img_rows = 28
img_cols = 28
num_features = 784

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = x_train[:num_examples], y_train[:num_examples]
x_test, y_test = x_test[:num_examples], y_test[:num_examples]
x_train = x_train.reshape(num_examples, num_features)
x_test = x_test.reshape(num_examples, num_features)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
train_labels_b = y_train
test_labels_b = y_test

print('x_train shape:', x_train.shape)
print('x_test shape:', x_train.shape)

x_train shape: (2000, 784)
x_test shape: (2000, 784)


First try to classify the data using a single layer feedforward neural network using batch gradient descent.

In [14]:
## Model (feedforward model)
model = Sequential() 

# Add a dense output layer where every node is connnected, and use softmax (pick the biggest mumber)
model.add(Dense(10, input_shape=(num_features,), activation='softmax')) 

## Cost function & Objective (and solver)

# Use stochastic gradient descent
sgd = optimizers.SGD(lr=0.02)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, train_labels_b, shuffle=False, batch_size=2000,verbose=0, epochs=52) 
score = model.evaluate(x_test, test_labels_b, verbose=0) 
print('Test accuracy:', score[1])

Test accuracy: 0.6154999732971191


Update batch gradient descent to stochastic gradient descent.

In [20]:
## Model
model = Sequential() 
model.add(Dense(10, input_dim=num_features, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.02)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, train_labels_b, shuffle=False, batch_size=100,verbose=0, epochs=50) 
score = model.evaluate(x_test, test_labels_b, verbose=0) 
print('Test accuracy:', score[1])

Test accuracy: 0.8215000033378601


# PART 2: Multi-layer Neural Networks

Take the implementation of logistic regression and add a hidden layer, making it a two layer neural network.  

In [22]:
## Model (feedforward) with 2 layers
model = Sequential() 

# One hidden layer
model.add(Dense(input_dim=num_features, units=20, activation='sigmoid')) 

# One output layer
model.add(Dense(input_dim=20, units=10, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.02)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=50) 
score = model.evaluate(x_test, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.4961794316768646
Test accuracy: 0.8460000157356262


#### Activation Revisted

In 2010, in a paper published at NIPS (https://www.utc.fr/~bordesan/dokuwiki/_media/en/glorot10nipsworkshop.pdf), Yoshua Bengio showed that rectifier activation works better empirically than sigmoid activation when used in the hidden layers.  

The rectifier activation is: f(x)=max(0,x).  Intuitively, the difference is that as a sigmoid activated node approaches 1 it stops learning even if error continues to be propagated to it, whereas the rectifier activated node continue to learn (at least in the positive direction).  Rectifiers also speed up training.

In [23]:
## Model
model = Sequential() 

# To prevent overfitting, we can use relu
model.add(Dense(units=30, input_dim=num_features, activation='relu')) 
model.add(Dense(units=10, input_dim=30, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.02)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=50) 
score = model.evaluate(x_test, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.520481288433075
Test accuracy: 0.8554999828338623


#### Noise

Previously when working with the MNIST data there was a benefit in generalization from adding noise to the training data (blurring). This time use a trick for adding noise called 'Dropouts'.  The idea with dropouts is that instead of (or in addition to) adding noise to our inputs, we add noise by having each node return 0 with a certain probability during training.  This trick both improves generalization in large networks and speeds up training.

Hinton introduced the idea in 2012 and gave an explanation of why it's similar to bagging (http://arxiv.org/pdf/1207.0580v1.pdf).

In [24]:
## Model
model = Sequential() 
model.add(Dense(units=30, input_dim=num_features, activation='relu')) 
model.add(Dropout(0.1))
model.add(Dense(units=10, input_dim=30, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=50) 
score = model.evaluate(x_test, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.45905208587646484
Test accuracy: 0.8644999861717224


# PART 3: Convolutional Neural Networks

Convolutional neural networks use different types of layers to create higher dimension abstractions/representations of data. First, reshape the data to be higher dimensional for the CNN.

In [25]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = x_train[:num_examples], y_train[:num_examples]
x_test, y_test = x_test[:num_examples], y_test[:num_examples]
x_train = x_train.reshape(num_examples, num_features)
x_test = x_test.reshape(num_examples, num_features)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_train.shape)

x_train shape: (2000, 28, 28, 1)
x_test shape: (2000, 28, 28, 1)


Add different convolutional layers with different sized kernels and a pooling layer.

In [6]:
model = Sequential() 

# Convolutional layers in 3D for spatial data
# We use a 3x3 kernel size (9 parameters per kernel)
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(img_rows, img_cols, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# Pooling is simply a summary statistic to make computation faster. 
# In this case we use max pooling (finding the max values)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Call flatten to go from 3D layers to 2D normally fully connected layers
model.add(Flatten())
model.add(Dense(units=50, input_dim=128, activation='relu')) 
model.add(Dense(units=10, input_dim=50, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Use a verbose fitting to show the computation during each epoch
history = model.fit(x_train, y_train, batch_size=10, epochs=100, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Epoch 1/100
200/200 [==============================] - 4s 16ms/step - loss: 2.0313 - accuracy: 0.3178 - val_loss: 0.8256 - val_accuracy: 0.7280
Epoch 2/100
200/200 [==============================] - 3s 14ms/step - loss: 0.7109 - accuracy: 0.7737 - val_loss: 0.6778 - val_accuracy: 0.7910
Epoch 3/100
200/200 [==============================] - 3s 14ms/step - loss: 0.4858 - accuracy: 0.8487 - val_loss: 0.5645 - val_accuracy: 0.8120
Epoch 4/100
200/200 [==============================] - 3s 13ms/step - loss: 0.3844 - accuracy: 0.8905 - val_loss: 0.5331 - val_accuracy: 0.8315
Epoch 5/100
200/200 [==============================] - 3s 14ms/step - loss: 0.3092 - accuracy: 0.9093 - val_loss: 0.4617 - val_accuracy: 0.8520
Epoch 6/100
200/200 [==============================] - 3s 14ms/step - loss: 0.3284 - accuracy: 0.9000 - val_loss: 0.4199 - val_accuracy: 0.8685
Epoch 7/100
200/200 [==============================] - 3s 16ms/step - loss: 0.2831 - accuracy: 0.9098 - val_loss: 0.3867 - val_accuracy: